<a href="https://colab.research.google.com/github/Lexiiz3417/sentimen-model.ipynb/blob/main/sentimen_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Persiapan (Install Library & Download Data NLTK)
Ini langkah awal buat install dan unduh semua yang dibutuhkan.

In [ ]:
!pip install pandas nltk scikit-learn tensorflow joblib
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import joblib
import nltk

# Unduh data stopwords Indonesia dari NLTK
nltk.download('punkt')
nltk.download('stopwords')

# 2. Muat & Bersihkan Data
Blok ini berfungsi buat ngambil data dari CSV dan ngebersihin teksnya.

In [ ]:
data = pd.read_csv('Corpus Bahasa Indonesia (Label).csv')

# Perbaikan untuk mengatasi KeyError: 'teks'
data.rename(columns={'kalimat': 'teks', 'sentiment': 'sentiment'}, inplace=True)

# Ganti nilai sentiment 2 menjadi 0 (Negatif/Netral)
# Jika ingin 3 kelas, jangan jalankan baris ini dan pastikan label y-nya 0, 1, 2
# data['sentiment'] = data['sentiment'].replace(2, 0)

list_stopwords = stopwords.words('indonesian')
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in list_stopwords]
    return ' '.join(filtered_tokens)

data['teks_bersih'] = data['teks'].apply(preprocess_text)

In [ ]:
# Cek nilai unik di kolom sentiment
print("\nNilai unik di kolom 'sentiment':")
print(data['sentiment'].unique())

# Cek tipe data dari kolom sentiment
print("\nTipe data kolom 'sentiment':")
print(data['sentiment'].dtype)

# 3. Ubah Teks ke Angka (TF-IDF)
Di sini kita ubah teks yang udah bersih jadi matriks angka.

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['teks_bersih'])

print("\nShape dari matriks TF-IDF:")
print(vectors.shape)

In [ ]:
# Kode untuk menyimpan vectorizer
!pip install joblib
import joblib
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

# 4. Bagi Data & One-Hot Encoding
Blok ini memisahkan data menjadi dua bagian, untuk diajarin dan dites.

In [ ]:
from tensorflow.keras.utils import to_categorical

# Ambil kolom sentiment sebagai label (y)
X = vectors
y = data['sentiment']

# Ubah label (0, 1, 2) menjadi one-hot encoding
y_one_hot = to_categorical(y)

# Bagi data menjadi 80% untuk latih dan 20% untuk uji
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

print("\nData berhasil dibagi dan diubah menjadi One-Hot Encoding!")
print(f"Jumlah data latih: {X_train.shape[0]}")
print(f"Jumlah data uji: {X_test.shape[0]}")
print(f"Bentuk label y_train: {y_train.shape}")


Data berhasil dibagi dan diubah menjadi One-Hot Encoding!
Jumlah data latih: 10087
Jumlah data uji: 2522
Bentuk label y_train: (10087, 3)


# 5. Bangun Model JST Multi-Kelas
Ini bagian inti di mana kita buat arsitektur JST-nya.

In [ ]:
# Bangun Model JST Sederhana
model = Sequential()

# Lapisan pertama
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))

# Lapisan output (diganti jadi 3 neuron dan softmax)
# 3 = jumlah kelas
# 'softmax' = fungsi aktivasi untuk multi-kelas
model.add(Dense(3, activation='softmax'))

# Tampilkan ringkasan model
model.summary()

# Kompilasi Model
# 'categorical_crossentropy' = fungsi loss untuk multi-kelas
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Latih Model
Blok terakhir ini buat 'ngajarin' model JST-mu pakai data latih.

In [ ]:
# Latih model
# 'epochs' adalah berapa kali model belajar dari seluruh data
# 'batch_size' adalah jumlah sampel per langkah pelatihan
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 7.  Simpan Model AI

In [ ]:
# Simpan model ke dalam file
model.save('sentimen_model.h5')

# Modelmu sekarang tersimpan dengan nama 'sentimen_model.h5' di Colab!
# Kamu bisa download file ini dari menu di samping kiri Colab